In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time
import torch
from torch_geometric.loader import DataLoader
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
# from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))
%matplotlib notebook
import copy

import time
import torch
from torch_geometric.loader import DataLoader
from tqdm.notebook import trange, tqdm
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))


import tomli
from scipy.optimize import minimize
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt

seed = 0


import random 
import numpy as np
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
# print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('running on: ', device)
torch.set_num_threads(1)

from joblib import Parallel, delayed

from cutlass import *
from rbfConv import *
from tqdm.notebook import tqdm

from datautils import *
# from sphUtils import *
from lossFunctions import *

In [2]:


"""Functions for reading the compressed training/validation data records"""
import os
import sys
import numpy as np
from glob import glob
import dataflow
import numpy as np
import zstandard as zstd
import msgpack
import msgpack_numpy
msgpack_numpy.patch()


class PhysicsSimDataFlow(dataflow.RNGDataFlow):
    """Data flow for msgpacks generated from SplishSplash simulations.
    """

    def __init__(self, files, random_rotation=False, shuffle=False, window=2):
        if not len(files):
            raise Exception("List of files must not be empty")
        if window < 1:
            raise Exception("window must be >=1 but is {}".format(window))
        self.files = files
        self.random_rotation = random_rotation
        self.shuffle = shuffle
        self.window = window

    def __iter__(self):
        decompressor = zstd.ZstdDecompressor()
        files_idxs = np.arange(len(self.files))
        if self.shuffle:
            self.rng.shuffle(files_idxs)

        for file_i in files_idxs:
            # read all data from file
            with open(self.files[file_i], 'rb') as f:
                data = msgpack.unpackb(decompressor.decompress(f.read()),
                                       raw=False)

            data_idxs = np.arange(len(data) - self.window + 1)
            if self.shuffle:
                self.rng.shuffle(data_idxs)

            # get box from first item. The box is valid for the whole file
            box = data[0]['box']
            box_normals = data[0]['box_normals']

            for data_i in data_idxs:

                if self.random_rotation:
                    angle_rad = self.rng.uniform(0, 2 * np.pi)
                    s = np.sin(angle_rad)
                    c = np.cos(angle_rad)
                    rand_R = np.array([c, 0, s, 0, 1, 0, -s, 0, c],
                                      dtype=np.float32).reshape((3, 3))

                if self.random_rotation:
                    sample = {
                        'box': np.matmul(box, rand_R),
                        'box_normals': np.matmul(box_normals, rand_R)
                    }
                else:
                    sample = {'box': box, 'box_normals': box_normals}

                for time_i in range(self.window):

                    item = data[data_i + time_i]

                    for k in ('pos', 'vel'):
                        if self.random_rotation:
                            sample[k + str(time_i)] = np.matmul(item[k], rand_R)
                        else:
                            sample[k + str(time_i)] = item[k]

                    for k in ('m', 'viscosity', 'frame_id', 'scene_id'):
                        sample[k + str(time_i)] = item[k]

                yield sample


def read_data_val(files, **kwargs):
    return read_data(files=files,
                     batch_size=1,
                     repeat=False,
                     shuffle_buffer=None,
                     num_workers=1,
                     **kwargs)


def read_data_train(files, batch_size, random_rotation=True, **kwargs):
    return read_data(files=files,
                     batch_size=batch_size,
                     random_rotation=random_rotation,
                     repeat=True,
                     shuffle_buffer=512,
                     **kwargs)


def read_data(files=None,
              batch_size=1,
              window=2,
              random_rotation=False,
              repeat=False,
              shuffle_buffer=None,
              num_workers=1,
              cache_data=False):
    print(files[0:20], '...' if len(files) > 20 else '')

    # caching makes only sense if the data is finite
    if cache_data:
        if repeat == True:
            raise Exception("repeat must be False if cache_data==True")
        if random_rotation == True:
            raise Exception("random_rotation must be False if cache_data==True")
        if num_workers != 1:
            raise Exception("num_workers must be 1 if cache_data==True")

    df = PhysicsSimDataFlow(
        files=files,
        random_rotation=random_rotation,
        shuffle=True if shuffle_buffer else False,
        window=window,
    )

    if repeat:
        df = dataflow.RepeatedData(df, -1)

    if shuffle_buffer:
        df = dataflow.LocallyShuffleData(df, shuffle_buffer)

    if num_workers > 1:
        df = dataflow.MultiProcessRunnerZMQ(df, num_proc=num_workers)

    df = dataflow.BatchData(df, batch_size=batch_size, use_list=True)

    if cache_data:
        df = dataflow.CacheData(df)

    df.reset_state()
    return df

['/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_00.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_01.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_02.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_03.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_04.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_05.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_06.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_07.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_00.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_01.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_02.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_03.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_04.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim

In [11]:

train_files = sorted(
    glob(os.path.join('/home/winchenbach/dev/datasets/WBCSPH2D', 'train', '*.zst')))

dataset = read_data_train(files=train_files,
                          batch_size=16,
                          window=3,
                          num_workers=1
                          )

df = PhysicsSimDataFlow(
    files=train_files,
    random_rotation=False,
    shuffle=False,
    window=3,
)

for frame in df:
#     print(frame.attrs.keys())
    print(frame.keys())
    print(frame['box'].shape)
    print(frame['box_normals'].shape)
    print(frame['pos0'].shape, frame['pos0'])
    print(frame['vel0'].shape, frame['vel0'])
    print(frame['m0'].shape, frame['m0'])
    print(frame['viscosity0'].shape, frame['viscosity0'])
    print(frame['frame_id0'])
    print(frame['scene_id0'])
    print(frame['pos1'].shape)
    print(frame['vel1'].shape)
    print(frame['m1'].shape)
    print(frame['viscosity1'].shape)
    print(frame['frame_id1'])
    print(frame['scene_id1'])
    print(frame['pos2'].shape)
    print(frame['vel2'].shape)
    print(frame['m2'].shape)
    print(frame['viscosity2'].shape)
    print(frame['frame_id2'])
    print(frame['scene_id2'])
    break

In [59]:

decompressor = zstd.ZstdDecompressor()
files_idxs = np.arange(len(train_files))

for file_i in files_idxs:
    # read all data from file
    with open(train_files[file_i], 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()),
                               raw=False)

    data_idxs = np.arange(len(data))

    # get box from first item. The box is valid for the whole file
    box = data[0]['box']
    box_normals = data[0]['box_normals']
    
    print(data)
    break
    

[{'box': array([[0.0025, 0.0025, 0.    ],
       [0.0075, 0.0025, 0.    ],
       [0.0025, 0.0075, 0.    ],
       ...,
       [0.9975, 0.9925, 0.    ],
       [0.9925, 0.9975, 0.    ],
       [0.9975, 0.9975, 0.    ]]), 'box_normals': array([[-0.70710678, -0.70710678, -0.        ],
       [-0.07741782, -0.99699874, -0.        ],
       [-0.99699874, -0.07741782, -0.        ],
       ...,
       [ 0.997402  ,  0.0720364 , -0.        ],
       [ 0.0720364 ,  0.997402  , -0.        ],
       [ 0.70710678,  0.70710678, -0.        ]]), 'frame_id': 0, 'scene_id': 'sim_0000', 'num_rigid_bodies': 0, 'pos': array([[0.5825, 0.0425, 0.    ],
       [0.5875, 0.0425, 0.    ],
       [0.5825, 0.0475, 0.    ],
       ...,
       [0.1575, 0.9525, 0.    ],
       [0.1525, 0.9575, 0.    ],
       [0.1575, 0.9575, 0.    ]]), 'vel': array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), 'm': array([[1.],
       [1.],
   

AttributeError: 'dict' object has no attribute 'attrs'

In [17]:
basePath = '../export'
basePath = os.path.expanduser(basePath)

simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.hdf5')]



2000


In [145]:
dist = 16
sublists = np.array_split(np.arange(frameCount // dist), 16 / dist)

inFile = h5py.File(simulationFiles[0], 'r')
frameCount = int(len(inFile['simulationExport'].keys()))

print(frameCount)

2000


In [146]:
box = np.float32(np.hstack((inFile['boundaryInformation']['boundaryPosition'], np.zeros((inFile['boundaryInformation']['boundaryPosition'].shape[0],1)))))
box_normals = np.float32(np.hstack((inFile['boundaryInformation']['boundaryNormals'], np.zeros((inFile['boundaryInformation']['boundaryNormals'].shape[0],1)))))

In [147]:
prefix = simulationFiles[0].split('/')[-1].split('.')[0]

s = 0
exportPath = '%s - %2d - %2d [%2d].msgpack.zst' % (prefix, dist, offset, s)

print(prefix)
print(exportPath)
if not os.path.exists('./convertedData/'):
    os.makedirs('./convertedData/')

dambreak - 2023-02-22_10-49-21
dambreak - 2023-02-22_10-49-21 - 16 - 15 [ 0].msgpack.zst


In [148]:

def create_compressed_msgpack(data, outfilepath):
    import zstandard as zstd
    import msgpack
    import msgpack_numpy
    msgpack_numpy.patch()

    compressor = zstd.ZstdCompressor(level=0)
    with open(outfilepath, 'wb') as f:
#         print('writing', outfilepath)
        f.write(compressor.compress(msgpack.packb(data, use_bin_type=True)))

In [149]:
from tqdm.notebook import tqdm

In [150]:

data = []
offset = 0
for offset in tqdm(range(dist)):
    for s, sl in enumerate(tqdm(sublists, leave = False)):
        outfilepath = './convertedData/%s - %02d - %02d [%02d].msgpack.zst' % (prefix, dist, offset, s)
#         print(outfilepath)
        data = []
#         print(sl)
        for si, ii in enumerate(sl):
            i = ii * dist + offset
            if i > len(inFile['simulationExport'].keys()):
                break
            frame = {}
            if si == 0:
#                 print(si)
                frame['box'] = box
                frame['box_normals'] = box_normals
    #         print(i)
        #     print(inFile['boundaryInformation'].keys())

        #     print(torch.from_numpy(inFile['boundaryInformation']['boundaryNormals'][:]))
            frameGroup = inFile['simulationExport']['%05d' % i]
            frame['pos']       = np.float32(np.hstack((frameGroup['fluidPosition'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['vel']       = np.float32(np.hstack((frameGroup['fluidVelocity'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['m']         = np.float32(np.expand_dims(frameGroup['fluidArea'][:], axis=1) * 998)
            frame['viscosity'] = np.float32(np.expand_dims(frameGroup['fluidArea'], axis=1) * 0)
            frame['frame_id']  = ii
            frame['scene_id']  = '%s - %2d - %2d' % (prefix, dist, offset)
            data.append(frame)
#         print(len(data))
    #         break
    #     print(data)
        create_compressed_msgpack(data, outfilepath)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                            | 0/124 [00:00<?, ?it/s]


In [151]:

train_files = sorted(
    glob(os.path.join('./convertedData/', 'train', '*.zst')))

dataset = read_data_train(files=train_files,
                          batch_size=16,
                          window=3,
                          num_workers=1
                          )

df = PhysicsSimDataFlow(
    files=train_files,
    random_rotation=False,
    shuffle=False,
    window=3,
)

for frame in df:
#     print(frame.attrs.keys())
    print(frame.keys())
    print(frame['box'].shape, frame['box'].dtype)
    print(frame['box_normals'].shape, frame['box_normals'].dtype)
    print(frame['pos0'].shape, frame['pos0'].dtype)
    print(frame['vel0'].shape, frame['vel0'].dtype)
    print(frame['m0'].shape, frame['m0'].dtype)
    print(frame['viscosity0'].shape, frame['viscosity0'].dtype)
    print(frame['frame_id0'])
    print(frame['scene_id0'])
    print(frame['pos1'].shape)
    print(frame['vel1'].shape)
    print(frame['m1'].shape)
    print(frame['viscosity1'].shape)
    print(frame['frame_id1'])
    print(frame['scene_id1'])
    print(frame['pos2'].shape)
    print(frame['vel2'].shape)
    print(frame['m2'].shape)
    print(frame['viscosity2'].shape)
    print(frame['frame_id2'])
    print(frame['scene_id2'])
    break

['./convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 00 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 01 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 02 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 03 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 04 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 05 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 06 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 07 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 08 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 09 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 10 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 11 [00].msgpack.zst', './